# Welcome to CSP solutions to vacation problem.

To define your own parameter in the CSP, please follow the cells below.

Set the variable `use_default` as false if different parameter are to be specified in this algorithm. 

Set it to `True` if the default values of all parameter are used, and immediately run all cells in this notebook.

In [0]:
use_default = False

Specify the absolute path to save the pickled solutions in the variable `pickled_solution_path`

In [0]:
pickled_solution_save_path = '/content/drive/My Drive/ai-ga-algo/csp-solutions-compressed.pyfile'

Specify the days of travelling in the variable `days_of_travel_param`.

Specify the allocated budget in the variable `allocated_budget_param`.

In [0]:
days_of_travel_param = 4
allocated_budget_param = 2500

## End of Parameter Specification 
Setting of parameter is done. Please run all of the cells below at once.

# Constraint Satisfaction Problem to Solve Vacation Problem

In [0]:
import constraint
import numpy as np

In [0]:
# saving to pickle functions
import pickle
import gzip

def dump_obj(obj, filename):
    with gzip.GzipFile(filename, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(filename):
    obj = None
    with gzip.GzipFile(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

## Code snippet for representation of more than one item (multi-hot encoding) ##
Useful for selecting multiple options for domain variables later

In [0]:
def add_choice(choices, **kwargs):
    index = 2 ** len(choices)
    choices[index] = kwargs
    #clear doubt here

def range_of_choices(choices):
    return range(sum(choices.keys()) + 1)
    #returns 2 ** len(choices)

def choose(choices, choice_number):
    power = len(choices)
    chosen_items = list()
 
    while choice_number > 0:
        if choice_number - 2**power >= 0:
            choice_number -= 2**power
            if 2**power in choices:
                chosen_items.append(choices[2**power])
        power -= 1
 
    return chosen_items

## Abbreviation for domain variables ##

In [0]:
abbr = {
    'h': 'hotels',
    'f': 'food_types',
    't': 'tourist_spots',
    'r': 'transportations'
}

## Create empty list for each domain variable ##

In [0]:
# To set up the domain space
hotels = list()
food_types = dict()
tourist_spots = dict()
transportations = dict()

## Insert attribute values for each domain variable ##

In [0]:
# Add hotels, e.g.
hotels.append({
    "name": "Avani Sepang Goldcoast Resort",
    "star": 5,
    "ppn": 470
})
    
hotels.append({
    "name": "Shangri-La Hotel Kuala Lumpur",
    "star": 5,
    "ppn": 504
})

hotels.append({
    "name": "St. Giles Hotel & Residences",
    "star": 4,
    "ppn": 412
})

hotels.append({
    "name": "Sunway Pyramid Hotel",
    "star": 4,
    "ppn": 302
})
    
hotels.append({
    "name": "Sunway Velocity Hotel",
    "star": 3,
    "ppn": 241
})
    
hotels.append({
    "name": "THE FACE Suites",
    "star": 3,
    "ppn": 349
})

In [0]:
# Add tourist spot, e.g.

add_choice(
    tourist_spots,
    name = "Sunway Lagoon",
    price = 108
)

add_choice(
    tourist_spots,
    name = "ATV Adventure Park",
    price = 200
)

add_choice(
    tourist_spots,
    name = "Garden of Nature",
    price = 121
)

add_choice(
    tourist_spots,
    name = "Petronas Twin Towers",
    price = 122
)

add_choice(
    tourist_spots,
    name = "Paradise Park Farm",
    price = 63
)



In [0]:
# Add food type, e.g.
add_choice(
    food_types,
    type = "Chinese Food",
    price = range(20, 30)
)

add_choice(
    food_types,
    type = "Indian Banana Leaf",
    price = range(10, 20)
)

add_choice(
    food_types,
    type = "Western Food",
    price = range(20, 50)
)


add_choice(
    food_types,
    type = "Local Seafood",
    price = range(20, 30)
)


add_choice(
    food_types,
    type = "Malay Food",
    price = range(7, 20)
)

In [0]:
# Add transportations, e.g.
add_choice(
    transportations,
    name = "Kommuter",
    ppd = 6
)


add_choice(
    transportations,
    name = "RapidKL Bus",
    ppd = 6
)


add_choice(
    transportations,
    name = "LRT",
    ppd = 5
)


add_choice(
    transportations,
    name = "KLIA Express",
    ppd = 55
)


add_choice(
    transportations,
    name = "Taxi",
    ppd = 40
)

## Defining the range of each variables ##

In [0]:
hotel_range = range(len(hotels))
food_type_range = range_of_choices(food_types)
tourist_spot_range = range_of_choices(tourist_spots)
transportation_range = range_of_choices(transportations)

## Fixed variable ##

In [0]:
# fixed variables
if use_default:
    days_of_travel = 4
    allocated_budget = 2500
else:
    days_of_travel = days_of_travel_param
    allocated_budget = allocated_budget_param

##  Constraint Function ##
The fixed variable is also introduced in constraint function

In [0]:
def constraint_func(h, t, f, r):
    hotel_cost = hotels[h]["ppn"] * days_of_travel

    tourist_list = choose(tourist_spots, t)
    tourist_cost = sum([i['price'] for i in tourist_list])

    food_list = choose(food_types, f)
    food_cost = sum([(max(i['price'])+min(i['price'])) / 2 * days_of_travel for i in food_list]) 
    
    transportation_list = choose(transportations, r)
    transportation_cost = sum([i['ppd'] * days_of_travel for i in transportation_list])
        
    total_cost = hotel_cost + tourist_cost + food_cost + transportation_cost
    return total_cost <= allocated_budget and len(tourist_list) >= 2 and len(food_list)> 0

In [0]:
def _constraint_func(**k):
    h, t, f, r = k['h'], k['t'], k['f'], k['r']
    hotel_cost = hotels[h]["ppn"] * days_of_travel

    tourist_list = choose(tourist_spots, t)
    tourist_cost = sum([i['price'] for i in tourist_list])

    food_list = choose(food_types,f)
    food_cost = sum([(max(i['price'])+min(i['price'])) / 2 * days_of_travel for i in food_list]) 
    
    transportation_list = choose(transportations,r)
    transportation_cost = sum([i['ppd'] * days_of_travel for i in transportation_list])
        
    total_cost = hotel_cost + tourist_cost + food_cost + transportation_cost
    return total_cost

## CSP

In [0]:
travelling_problem = constraint.Problem()

travelling_problem.addVariables('h', hotel_range)
travelling_problem.addVariables('t', tourist_spot_range)
travelling_problem.addVariables('f', food_type_range)
travelling_problem.addVariables('r', transportation_range)

travelling_problem.addConstraint(constraint_func, list(['h', 't', 'f', 'r']))

## Extract solutions of CSP ##

In [0]:
solutions = travelling_problem.getSolutions()

In [209]:
print(len(solutions))

96643


## Saving solutions using pickle

In [0]:
# pickle the solutions to be used in GA python notebook
if use_default:
    path = '/content/drive/My Drive/ai-ga-algo/'
    dump_obj(solutions, path + 'csp-solutions-compressed.pyfile')

else:
    dump_obj(solutions, pickled_solution_save_path)

## Key to sort solution in ascending order ##

In [0]:
def mysort(x):
    hotel_cost = hotels[x['h']]["ppn"] * days_of_travel

    tourist_list = choose(tourist_spots, x['t'])
    tourist_cost = sum([i['price'] for i in tourist_list])

    food_list = choose(food_types, x['f'])
    food_cost = sum([(max(i['price'])+min(i['price'])) / 2 * days_of_travel for i in food_list]) 
    
    transportation_list = choose(transportations, x['r'])
    transportation_cost = sum([i['ppd'] * days_of_travel for i in transportation_list])

    total_cost = hotel_cost + tourist_cost + food_cost + transportation_cost
    return total_cost 

## Sort the solution in ascending order of total_cost ##

In [0]:
sorted_solutions = sorted(solutions, key = mysort)

## Listing out the top 5 solutions ##

In [213]:
for index, x in enumerate(sorted_solutions[:5]):
    print("--------------------------------------------------")
    print("Option",index+1,":")
    temp = hotels[x['h']]
    print("Hotel name: {} \nHotel star: {} \nHotel price per night: RM{}".format(temp["name"], temp["star"], temp["ppn"]))
    print("Total Hotel cost for",days_of_travel,"days --> RM", temp["ppn"] * days_of_travel,"\n")
    
    food_list = choose(food_types, x['f'])
    print("Number of food types to eat:", len(food_list))
    for i,j in enumerate(food_list):
        print("Food choice", i+1 )
        print("Food type: {} \nFood price range: RM{}-{}\n".format(j["type"], j["price"][0], 1+j["price"][-1]))
    print("Total food cost for", days_of_travel, "days --> RM", sum([(max(i['price'])+min(i['price'])) / 2 * days_of_travel for i in food_list]),"\n" )
    
    tourist_list = choose(tourist_spots, x['t'])
    print("Number of tourist spot to visit:", len(tourist_list))
    for i,j in enumerate(tourist_list):
        print("Tourist spot choice", i+1 )
        print("Place name: {} \nEntrance price: RM{}\n".format(j["name"], j["price"]))    
    print("Total tourism cost --> RM", sum([i['price'] for i in tourist_list]),"\n" )
    
    transportation_list = choose(transportations, x['r'])
    print("Number of transportation to use:", len(transportation_list))
    for i,j in enumerate(transportation_list):
        print("Transportation choice", i+1 )
        print("Transportation name: {} \nTransportation price per day: RM{}\n".format(j["name"], j["ppd"]))    
    print("Total transportation cost for", days_of_travel, "days --> RM", sum([i['ppd'] for i in transportation_list])*days_of_travel,"\n" )
    
    print("Overall total cost for option",index+1,"--> RM",mysort(x))

--------------------------------------------------
Option 1 :
Hotel name: Sunway Velocity Hotel 
Hotel star: 3 
Hotel price per night: RM241
Total Hotel cost for 4 days --> RM 964 

Number of food types to eat: 1
Food choice 1
Food type: Malay Food 
Food price range: RM7-20

Total food cost for 4 days --> RM 52.0 

Number of tourist spot to visit: 2
Tourist spot choice 1
Place name: Paradise Park Farm 
Entrance price: RM63

Tourist spot choice 2
Place name: Sunway Lagoon 
Entrance price: RM108

Total tourism cost --> RM 171 

Number of transportation to use: 0
Total transportation cost for 4 days --> RM 0 

Overall total cost for option 1 --> RM 1187.0
--------------------------------------------------
Option 2 :
Hotel name: Sunway Velocity Hotel 
Hotel star: 3 
Hotel price per night: RM241
Total Hotel cost for 4 days --> RM 964 

Number of food types to eat: 1
Food choice 1
Food type: Indian Banana Leaf 
Food price range: RM10-20

Total food cost for 4 days --> RM 58.0 

Number of tou